In [91]:
from UTILS import Time_Converter, Geographic
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

In [2]:
df_1 = pd.read_csv("geographic.csv")
df_2 = pd.read_csv("mta_key.csv")

In [ ]:
df_3 = pd.read_csv("nyc_bikeshare.csv")

In [ ]:
df = df[df['birthyear'] >=  1935]
df = df.assign(age = 2019 - df['birthyear'])

df['under_20'] = [1 if x<20 else 0 for x in df['age']]
df['20-40'] = [1 if x<41 and x>19 else 0 for x in df['age']]
df['40-64'] = [1 if x<65 and x>40 else 0 for x in df['age']]
df['over_65'] = [1 if x>=64 else 0 for x in df['age']]

In [12]:
null_con = df_3.isnull().sum()
null_con = null_con.to_frame(name = "Missing")
null_con["rate"] = null_con["Missing"]/df_3.shape[0]

In [16]:
null_con.to_csv("Missing Condition.csv", index = True)

In [ ]:
df_3_1 = df_3.dropna(axis = 0, subset = ["start_station_id", "end_station_id", "birthyear"])
df_3_2 = df_3_1.loc[df_3_1["gender"] != 0, :]
del df_3, df_3_1

In [100]:
df_4 = pd.read_csv("nyc_bikeshare_key.csv")
df_4.shape

(1105, 4)

In [50]:
def data_type_converter(data, idx):
    data[idx] = data[idx].astype("int")
    
for i in ("start_station_id", "end_station_id"):
    data_type_converter(df_3_2, i)

In [62]:
df_3_2["starttime"] = df_3_2["starttime"].apply(lambda x: Time_Converter.Date_Parser_2(x))
df_3_2["stoptime"] = df_3_2["stoptime"].apply(lambda x: Time_Converter.Date_Parser_2(x))

In [3]:
df_3_2 = pd.read_pickle("nyc_bikeshare_1.pickle")

In [18]:
print(df_3_2.shape)

(24410012, 9)


In [4]:
thre = 10800
df_3_2 = df_3_2.loc[df_3_2["tripduration"] <= 10800]

In [21]:
df_3_2.shape[0]/24410012

0.9988549370643488

In [24]:
thre_25 = df_3_2[["tripduration"]].quantile(0.25)
thre_50 = df_3_2[["tripduration"]].quantile(0.5)
thre_75 = df_3_2[["tripduration"]].quantile(0.75)
thre_100 = df_3_2[["tripduration"]].quantile(1)

# df_3_2[["tripduration"]].boxplot()

In [25]:
def Duration_Separation(data, low, high):
    data = data.loc[data["tripduration"].apply(lambda x: x > low and x < high)]
    return data

sep_1 = Duration_Separation(df_3_2, 0, int(thre_25))
gender_sep_thre_25 = sep_1.groupby("gender").count()
sep_2 = Duration_Separation(df_3_2, int(thre_25), int(thre_50))
gender_sep_thre_50 = sep_2.groupby("gender").count()
sep_3 = Duration_Separation(df_3_2, int(thre_50), int(thre_75))
gender_sep_thre_75 = sep_3.groupby("gender").count()
sep_4 = Duration_Separation(df_3_2, int(thre_75), int(thre_100))
gender_sep_thre_100 = sep_4.groupby("gender").count()

In [52]:
def M_F_Ratio_by_year(data, year):
    M_F = data.loc[data["starttime"].dt.year == year].groupby("gender").count()["tripduration"].values 
    return M_F[0]/M_F[1]

dyct = {}
for e, i in enumerate([sep_1, sep_2, sep_3, sep_4]):
    inner_dyct = {}
    for j in (2013, 2014, 2015, 2016, 2017, 2018, 2019):
        inner_dyct[str(j)] = M_F_Ratio_by_year(i, j)
        dyct[str(e)] = inner_dyct
        
M_F_R = pd.DataFrame.from_dict(dyct).rename(columns = {0: "0_25", 1: "25_50", 2: "50_75", 3: "75_100"})
M_F_R.to_csv("Male_to_Female_ratio.csv")    

In [29]:
dyct = {"From 0 to 25": gender_sep_thre_25,
        "From 25 to 50": gender_sep_thre_50, 
        "From 50 to 75": gender_sep_thre_75, 
        "From 75 to 100": gender_sep_thre_100}

import pickle 
with open('Duration_Separation_Gender.pickle', 'wb') as handle:
    pickle.dump(dyct, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df_3_2["Weekday"] = df_3_2["starttime"]

In [11]:
df_3_2["weekday"] = df_3_2["starttime"].dt.dayofweek

In [15]:
df_weekday = df_3_2.loc[df_3_2["weekday"].apply(lambda x: x in (0,1,2,3,4))]

In [18]:
df_weekend = df_3_2.loc[df_3_2["weekday"].apply(lambda x: x in (5,6))]

In [20]:
df_weekday.to_pickle("weekday.pickle")
df_weekend.to_pickle("weekend.pickle")

In [67]:
df_weekday.groupby("gender").count()

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid,usertype,birthyear,weekday
gender,,,,,,,,,
1,8710398,8710398,8710398,8710398,8710398,8710398,8706773,8710398,8710398
2,2733360,2733360,2733360,2733360,2733360,2733360,2731652,2733360,2733360


In [68]:
df_weekend.groupby("gender").count()

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid,usertype,birthyear,weekday
gender,,,,,,,,,
1,3873661,3873661,3873661,3873661,3873661,3873661,3869637,3873661,3873661
2,1541879,1541879,1541879,1541879,1541879,1541879,1539571,1541879,1541879


In [71]:
dyct_ = {}
for e, i in enumerate([df_weekday, df_weekend]):
    inner_dyct = {}
    for j in (2013, 2014, 2015, 2016, 2017, 2018, 2019):
        inner_dyct[str(j)] = M_F_Ratio_by_year(i, j)
        dyct_[str(e)] = inner_dyct
        
M_F_R = pd.DataFrame.from_dict(dyct_)

In [81]:
M_F_R.to_csv("Weekday_Weekend_M_F_Ratio.csv")

In [82]:
df_3_1 = pd.read_csv("nyc_bikeshare_key.csv")

In [85]:
df_3_2["station_id"] = df_3_2["start_station_id"]
df_3_3 = df_3_2.merge(df_3_1, how = "outer", on = "station_id").dropna(subset = ["tripduration"], axis = 0)

In [94]:
K_clusters = [10]
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = df_3_3[["station_latitude"]]
X_axis = df_3_3[["station_longitude"]]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]

KeyboardInterrupt: 